In [1]:

!cd python-telegram-bot
!pip3 install python-telegram-bot
!pip3 install python-telegram-bot --upgrade pip

Requirement already up-to-date: python-telegram-bot in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (12.8)
Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (20.2.2)


In [2]:
! pip3 install requests

In [320]:
import logging
from telegram.ext import Updater, InlineQueryHandler, CommandHandler, MessageHandler, Filters, RegexHandler
import requests
import re
import random
import telegram



In [321]:

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
logger = logging.getLogger(__name__)

In [322]:
def help_command(bot,update):
    """Send a message when the command /help is issued."""
    bot.send_message(chat_id=update.message.chat_id, text='Do you have any problem?')

In [323]:
keywords = {
            'greet': ['hello', 'hi', 'hey','name','call','Hello','Hi'], 
            'thankyou': ['thank', 'thx','Thank'], 
            'goodbye': ['bye', 'farewell','Bye']
           }
# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    
    # Create regular expressions and compile them into pattern objects
    patterns[intent] =re.compile('|'.join(keys))

In [324]:
responses = {
             'thankyou': 'you are very welcome',  
             'goodbye': 'goodbye for now',
             
            }
# Define a function to find the intent of a message
def match_intent(message):
    message.lower()
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        
        if re.search(pattern,message):
            matched_intent = intent
        
    return matched_intent

In [325]:
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword= re.compile("name|call")
    # Create a pattern for finding capitalized words
    name_pattern = re.compile("[A-Z]{1}[a-z]*")
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

In [326]:
import requests
def search(message):
    aString = ""
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/detect"
    payload = "text=" + message
    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505",
        'content-type': "application/x-www-form-urlencoded"
    }

    response = requests.post(url, data=payload, headers=headers).json()
    for i in response["annotations"]:
        aString = aString + i["annotation"] +", "
    return aString

In [327]:
def search1(message):
    aString = ""
    for i in search(message):
        if i !=",":
            aString = aString + i
    return aString

In [328]:
def search2(message):
    aString = ""
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/detect"
    payload = "text=" + message
    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505",
        'content-type': "application/x-www-form-urlencoded"
    }

    response = requests.post(url, data=payload, headers=headers).json()
    for i in response["annotations"]:
        if i["tag"] == "dish":
            aString += i["annotation"]
    return aString

In [329]:
def query(message):
    aString = ""
    for i in search(message):
        if i != ",":
            aString = aString + i
    return aString


In [330]:
def getNum(message):
    aString =""
    for i in message:
        if i.isdigit():
            aString = aString + i
    return aString

In [331]:
def getDiet(message):
    aString = ""
    if "vegan" in message:
        aString = "vegan"
    elif "vegetarian" in message:
        aString = "vegetarian"
    elif "paleo" in message:
        aString = "paleo"
    return aString


In [332]:

def generatemealplan(message):
    if getNum(message) == "":
        num = "2000"
    else:
        num = getNum(message)
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/mealplans/generate"
    querystring = {"timeFrame":"day","targetCalories":num,"diet":getDiet(message),"exclude":search(message)}
    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
    }
    response = requests.get(url, headers=headers, params=querystring).json()
    return response

In [333]:
line = "Welcome to the HappyChatter, I'm Shira, glad to see you and serve you.\nNow I will take a few time to introduce my function to you.\n(1) There are 4 buttons, for the first button, you can give me some special needs like you hate which food, your calories limits in one day, or your habit for eating. \n(2) For the second button, you can find the video about cook, and learn how to cook. \n(3) For the third button, give me a name for a receipt, I can give you its nutrients contain calories,protein \n(4) For the forth button, you can ask me some questions or just chat with me. \nHope that you have a good time "
def start(bot, update):
    kb = [[telegram.KeyboardButton('Generate the meal plan')],
          [telegram.KeyboardButton('Find the cook video')],
         [telegram.KeyboardButton("Analyze the nutrients")],
         [telegram.KeyboardButton("Happy chat")]]
    kb_markup = telegram.ReplyKeyboardMarkup(kb)
    bot.send_message(chat_id=update.message.chat_id,text=line,reply_markup=kb_markup)



In [334]:
aList = []
def hide(bot,update):
    if "Generate the meal plan" in update.message.text:
        aList.append("Generate the meal plan")
        bot. send_message(chat_id=update.message.chat_id, text = "Befor give you my advice, I would like to ask do you have any special needs that I should pay attention on",reply_markup = telegram.ReplyKeyboardRemove())
    if "Find the cook video" in update.message.text:
        aList.append("Find the cook video")
        bot. send_message(chat_id=update.message.chat_id, text = "There are a lot of videos, I'm confused which dishes you would like to find the relative video",reply_markup = telegram.ReplyKeyboardRemove())
    if "Analyze the nutrients" in update.message.text:
        aList.append("Analyze the nutrients")
        bot. send_message(chat_id=update.message.chat_id, text = "Give me a specific name for the receipe that you want to have the specific information, I will analyze the nutrients for you",reply_markup = telegram.ReplyKeyboardRemove())
    if "Happy chat" in update. message.text:
        aList.append("Happy chat")
        bot. send_message(chat_id=update.message.chat_id, text = "Happy to see you!!! Do you have any question or any thoughts that want to chat with me",reply_markup = telegram.ReplyKeyboardRemove())

In [335]:
def searchvideo(message):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/videos/search"

    querystring = {"query":query(message),"excludeingredients":"mustard","includeingredients":search(message),"minLength":"0","maxLength":"999","offset":"0","number":"10"}

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
        }

    response = requests.get(url, headers=headers, params=querystring).json()
    num = random.randint(0,5)
    aString = "https://www.youtube.com/watch?v=" + response["videos"][num]["youTubeId"]
    return aString


In [336]:
def find(message):
    aString = ""

    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/guessNutrition"

    querystring = {"title":message}

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
        }

    response = requests.get(url, headers=headers, params=querystring).json()
    aString = aString +"calories: "+str(response["calories"]["value"]) +" " + str(response["calories"]["unit"]) + ", fat: " + str(response["fat"]["value"]) + " " +str(response["fat"]["unit"])+", protein: "+str(response["protein"]["value"])+" "+str(response["protein"]["unit"])+", carbs: "+str(response["carbs"]["value"]) +" "+str(response["carbs"]["unit"])
    return aString

In [337]:
def joke():
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/jokes/random"

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
        }

    response = requests.get(url, headers=headers).json()

    return response["text"]

In [338]:
def fact():
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/trivia/random"

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
        }

    response = requests.get(url, headers=headers).json()

    return response["text"]


In [339]:
def quick(message):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/quickAnswer"

    querystring = {"q":message}

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
        }

    response = requests.get(url, headers=headers, params=querystring).json()

    return response["answer"]

In [340]:
def recommend(message):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/wine/recommendation"

    querystring = {"maxPrice":"50","minRating":"0.7","number":"3","wine":search1(message)}

    headers = {
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
        'x-rapidapi-key': "c4b855b7dfmshbc032041d307865p10b87ejsn8e37aa9ca505"
    }

    response = requests.get(url, headers=headers, params=querystring).json()

    return response["recommendedWines"][0]

In [341]:
def mode1(bot,update):
    if "change" in update.message.text:
        bot.send_message(chat_id=update.message.chat_id, text="This is another one, if you have any other special needs or you want to change again,you can tell me ")
        bot.send_message(chat_id=update.message.chat_id, text=respond1(update.message.text))
    elif "don't" in update.message.text or "calories" in update.message.text or "vegan" in update.message.text or "paleo" in update.message.text:
        bot.send_message(chat_id=update.message.chat_id, text="Below is the advice, hope that you can love it")
        bot.send_message(chat_id=update.message.chat_id, text=respond1(update.message.text))
    

In [342]:
def mode2(bot,update):
    if search(update.message.text)!="":
        bot.send_message(chat_id=update.message.chat_id, text="This is the video, hope that it can help you")
        bot.send_message(chat_id=update.message.chat_id, text=searchvideo(update.message.text),supports_streaming=True)

In [343]:
def mode3(bot,update):
    if "name" in update.message.text: 
        bot.send_message(chat_id=update.message.chat_id, text=find(update.message.text[5:]))

In [344]:
def mode4(bot,update):
    if "joke" in update.message.text:
        bot.send_message(chat_id=update.message.chat_id, text="this is a joke that I find for you, hope that you will have a good mood")
        bot.send_message(chat_id=update.message.chat_id, text=joke())
    if "fact" in update.message.text:
        bot.send_message(chat_id=update.message.chat_id, text="There are a lot of amazing things about food, if you want hear, I can tell you more like this")
        bot.send_message(chat_id=update.message.chat_id, text=fact())
    if "?" in update.message.text and search(update.message.text)!="":
        bot.send_message(chat_id=update.message.chat_id, text="Hope that this answer can be helpful for your question")
        bot.send_message(chat_id=update.message.chat_id, text=quick(update.message.text))
    if search2(update.message.text) != "":
        bot.send_message(chat_id=update.message.chat_id, text="This is my recommendation, incuding its title, description and link, hope that you like it")
        bot.send_message(chat_id=update.message.chat_id, text=recommend(update.message.text)["title"])
        bot.send_message(chat_id=update.message.chat_id, text=recommend(update.message.text)["description"])
        bot.send_message(chat_id=update.message.chat_id, text=recommend(update.message.text)["link"])

In [345]:
import random
def respond(bot,update):
    # Call the match_intent function
    
    intent = match_intent(update.message.text)
    # Fall back to the default response
    key =""
    if intent in responses:
        key = intent
    name = find_name(update.message.text)
    hide(bot,update)
    if "change" in update.message.text and "mode" in update.message.text:
        start(bot,update)
    elif aList[-1] == "Generate the meal plan":
        mode1(bot,update)
    elif aList[-1] == "Find the cook video":
        mode2(bot,update)
    elif aList[-1] == "Analyze the nutrients":
        mode3(bot,update)
    elif aList[-1] == "Happy chat":
        mode4(bot,update)
    if key == "greet" or key == 'thankyou' or key =='goodbye':
        bot.send_message(chat_id=update.message.chat_id, text=responses[key])

    

    

In [346]:
 def respond1(message):
        response=generatemealplan(message)
        return response["meals"] [random.randint(0,2)]["sourceUrl"]

In [347]:
updater = Updater('1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8')
dp = updater.dispatcher
dp.add_handler(CommandHandler("start", start))
dp.add_handler(CommandHandler("help", help_command))
dp.add_handler(MessageHandler(Filters.text, respond))


/Users/shira/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


In [348]:
updater.start_polling()

2020-09-08 22:18:19,514 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connectionpool.HTTPSConnectionPool object at 0x1054f30d0>, 'Connect timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getMe


2020-09-08 22:52:50,967 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/ext/dispatcher.py", line 340, in process_update
    handler.handle_update(update, self, check, context)
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/ext/handler.py", line 122, in handle_update
    return self.callback(dispatcher.bot, update, **optional_args)
  File "<ipython-input-345-b8c771de63ef>", line 21, in respond
    mode4(bot,update)
  File "<ipython-input-344-7bcc01fad36d>", line 10, in mode4
    bot.send_message(chat_id=update.message.chat_id, text=quick(update.message.text))
  File "<ipython-input-339-d31bd0332586>", line 13, in quick
    return response["answer"]
KeyError: 'answer'
2020-09-08 22:59:25,579 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after conne

2020-09-08 22:59:51,616 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1055383d0>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 22:59:56,619 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1055df2d0>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:00:01,624 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegr

2020-09-08 23:00:23,167 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 617, in urlopen
    chunked=chunked)
  File "/Users/shira/.local/lib/python3.7/site-packages/teleg

2020-09-08 23:00:53,833 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x105538310>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:00:58,838 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x105538410>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:01:03,841 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, r

...

2020-09-08 23:03:10,141 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x105523e10>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2020-09-08 23:03:10,142 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/shira/.local/lib/python3.7/site-packages/telegr

2020-09-08 23:04:39,254 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x105538390>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:04:44,263 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x10544ea10>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:04:49,267 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, r

2020-09-08 23:05:44,308 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1055dfa90>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2020-09-08 23:05:44,309 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/shira/.local/lib/python3.7/site-packages/telegr

2020-09-08 23:21:36,756 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1055f17d0>: Failed to establish a new connection: [Errno 65] No route to host')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:21:36,759 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1055f1310>: Failed to establish a new connection: [Errno 65] No route to host')': /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates
2020-09-08 23:21:36,762 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None,

2020-09-08 23:22:16,788 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1373531906:AAEZ4ELQ3uqQirlfYwk0clguiBUluvOQxE8/getUpdates (Caused by NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x1054b1ed0>: Failed to establish a new connection: [Errno 65] No route to host'))
2020-09-08 23:22:16,789 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/shira/.local/lib/python3.7/site-packages/telegram/vendor/ptb_urllib3/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/shira/.local/lib/python3.7/site-packages/teleg

In [319]:
updater.stop()